## A tool for sorting MOSS results
* Replace `my_url` in the first cell with the url of the results from MOSS server
* `Moss_Sorted.html` will be created that contains the sorted results based on maximum matched percent in the first two columns. 

In [150]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd

my_url="my_url"
pg1=rq.get(my_url)
page_1=BeautifulSoup(pg1.text,'html.parser')
my_df=pd.read_html(pg1.text)[0] #contains the table from original MOSS results

table = page_1.find('table')
links = []

#taken and modified from https://stackoverflow.com/questions/56757261/extract-href-using-pandas-read-html
for tr in table.findAll("tr"):
    trs = tr.findAll("td")
    link1=[]
    for each in trs:
        try:
            link = each.find('a')['href']
            #print(link)
            link1.append(link)
        except:
            pass
    links.append(link1)


links=links[1:]
final_links=[link[0] for link in links] #contains one link per row 
my_df["Link"]=final_links #adds links column to the data fram
#print(final_links)

for index,row in my_df.iterrows():
    a=row["File 1"].split(" ")[1].split("%")[0].split("(")[1]
    b=row["File 2"].split(" ")[1].split("%")[0].split("(")[1]
    my_df.loc[index,"Max_Percent"]=max(int(a),int(b)) #adds a column for max_percent

my_df=my_df.sort_values(by=["Max_Percent"],ascending=False)#sort based on max percent

#The function is taken and modefied from https://discuss.streamlit.io/t/display-urls-in-dataframe-column-as-a-clickable-hyperlink/743/4
def make_clickable(link):
    text = link.split('/')[-1]
    return f'<a target="_blank" href="{link}">{text}</a>' #puts each link in <a> tag so that the converted version to html looks like a link

for index,row in my_df.iterrows():
    a=make_clickable(row["Link"])
    my_df.loc[index,"Link"]=a #puts each link in <a> tag so that the converted version to html looks like a link
md_df1=my_df.to_html('Moss_Sorted.html',escape=False) #export the data frame to an html file
